## FERC <> EIA Granular Connections

Notes on the type of problem we are trying to solve:
- A classification problem
    - A Multi-Class Classification problem*
- A deterministic problem
- A record linkage problem

Right now, we are using the recordlinkage package. We're using logistic regression classifier because it fits all of the above.

To consider:
- Maybe we want to run the records with fuel cost data through a different matching model...

In [8]:
# load new MUL
make_plant_parts_eia.get_master_unit_list_eia(file_path_mul, clobber=True)

Master unit list not found /Users/aesharpe/Desktop/Work/Catalyst_Coop/rmi-ferc1-eia/outputs/master_unit_list.pkl.gzGenerating a new master unit list. This should take ~10 minutes.
Generating the master generator table with ownership.
beginning the aggregation for generation_eia923
getting generation_eia923 from the sqlite db
beginning the aggregation for generators_eia860
getting generators_eia860 from the sqlite db
beginning the aggregation for mcoe
getting mcoe from the output object
filling in fuel cost NaNs EIA APIs monthly state averages
filling in fuel cost NaNs with rolling averages
getting ownership_eia860 from the sqlite db
getting boiler_generator_assn_eia860 from the sqlite db
getting generators_entity_eia from the sqlite db
true grans found for plant: 126635
true grans found for plant_unit: 130529
true grans found for plant_prime_mover: 40168
true grans found for plant_technology: 70384
true grans found for plant_prime_fuel: 7149
true grans found for plant_ferc_acct: 47896


plant_id_eia report_date plant_part  \
record_id_eia                                                                 
1_2019_plant_owned_63560                           1  2019-01-01      plant   
1_2019_plant_total_63560                           1  2019-01-01      plant   
2_2019_plant_owned_195                             2  2019-01-01      plant   
2_2019_plant_total_195                             2  2019-01-01      plant   
3_2019_plant_owned_195                             3  2019-01-01      plant   
...                                              ...         ...        ...   
62938_39001_2018_plant_gen_total_62792         62938  2018-01-01  plant_gen   
62939_41001_2018_plant_gen_owned_62791         62939  2018-01-01  plant_gen   
62939_41001_2018_plant_gen_total_62791         62939  2018-01-01  plant_gen   
62940_pv_2018_plant_gen_owned_62764            62940  2018-01-01  plant_gen   
62940_pv_2018_plant_gen_total_62764            62940  2018-01-01  plant_gen   

                                       generator_id unit_id_pudl  \
record_id_eia                                                      
1_2019_plant_owned_63560                        NaN         <NA>   
1_2019_plant_total_63560                        NaN         <NA>   
2_2019_plant_owned_195                            1         <NA>   
2_2019_plant_total_195                            1         <NA>   
3_2019_plant_owned_195                          NaN         <NA>   
...                                             ...          ...   
62938_39001_2018_plant_gen_total_62792        39001         <NA>   
62939_41001_2018_plant_gen_owned_62791        41001         <NA>   
62939_41001_2018_plant_gen_total_62791        41001         <NA>   
62940_pv_2018_plant_gen_owned_62764              PV         <NA>   
62940_pv_2018_plant_gen_total_62764              PV         <NA>   

                                       prime_mover_code energy_source_code_1  \
record_id_eia                                                                  
1_2019_plant_owned_63560                            NaN                 <NA>   
1_2019_plant_total_63560                            NaN                 <NA>   
2_2019_plant_owned_195                               HY                  WAT   
2_2019_plant_total_195                               HY                  WAT   
3_2019_plant_owned_195                              NaN                 <NA>   
...                                                 ...                  ...   
62938_39001_2018_plant_gen_total_62792               WT                  WND   
62939_41001_2018_plant_gen_owned_62791               WT                  WND   
62939_41001_2018_plant_gen_total_62791               WT                  WND   
62940_pv_2018_plant_gen_owned_62764                  PV                  SUN   
62940_pv_2018_plant_gen_total_62764                  PV                  SUN   

                                            technology_description  \
record_id_eia                                                        
1_2019_plant_owned_63560                                       NaN   
1_2019_plant_total_63560                                       NaN   
2_2019_plant_owned_195                  Conventional Hydroelectric   
2_2019_plant_total_195                  Conventional Hydroelectric   
3_2019_plant_owned_195                                         NaN   
...                                                            ...   
62938_39001_2018_plant_gen_total_62792        Onshore Wind Turbine   
62939_41001_2018_plant_gen_owned_62791        Onshore Wind Turbine   
62939_41001_2018_plant_gen_total_62791        Onshore Wind Turbine   
62940_pv_2018_plant_gen_owned_62764             Solar Photovoltaic   
62940_pv_2018_plant_gen_total_62764             Solar Photovoltaic   

                                       ferc_acct_name  utility_id_eia  \
record_id_eia                                                           
1_2019_plant_owned_63560     

In [201]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [202]:
import pandas as pd
import numpy as np
import pudl
import pudl.constants as pc
import pudl.extract.ferc1
import sqlalchemy as sa
import logging
import sys
import copy
from copy import deepcopy
import scipy
import statistics
import yaml

import recordlinkage as rl
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [203]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [204]:
sys.path.append("../")
from pudl.output.ferc1 import *
from pudl_rmi.connect_ferc1_to_eia import *
from pudl_rmi.make_plant_parts_eia import *
import pudl_rmi.connect_ferc1_to_eia
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
ferc_engine = sa.create_engine(pudl_settings['ferc1_db'])
pd.options.display.max_columns = None

### Pull in data (EIA, FERC and training)

In [205]:
file_path_training = pathlib.Path().cwd().parent /'inputs'/'train_ferc1_to_eia.csv'
file_path_mul = pathlib.Path().cwd().parent /'outputs' /'master_unit_list.pkl.gz'
# pudl output object for ferc data
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine, ferc_engine, freq='AS',fill_fuel_cost=True,roll_fuel_cost=True,fill_net_gen=False)

In [238]:
%%time
inputs = InputManager(file_path_training, file_path_mul, pudl_out)
features_all = (Features(feature_type='all', inputs=inputs)
                .get_features(clobber=False))
features_train = (Features(feature_type='training', inputs=inputs)
                  .get_features(clobber=False))
tuner = ModelTuner(features_train, inputs.get_train_index(), n_splits=10)

matcher = MatchManager(best=tuner.get_best_fit_model(), inputs=inputs)
matches_best = matcher.get_best_matches(features_train, features_all)

Preparing the FERC1 tables.
Reading the master unit list from /Users/aesharpe/Desktop/Work/Catalyst_Coop/rmi-ferc1-eia/outputs/master_unit_list.pkl.gz
Generated 136066 all candidate features.
Generated 543 training candidate features.
We are about to test hyper parameters of the model while doing k-fold cross validation. This takes a few minutes....
Scores from the best model hyperparameters:
  F-Score:   0.79
  Precision: 0.88
  Accuracy:  0.54
Fit and predict a model w/ the highest scoring hyperparameters.
Get the top scoring match for each FERC1 steam record.
Winning match stats:
        matches vs ferc:      64.13%
        best match v ferc:    55.80%
        best match vs matches:87.00%
        murk vs matches:      0.80%
        ties vs matches:      7.75%
Overridden records:       9.6%
New best match v ferc:    56.10%
CPU times: user 2min 43s, sys: 12.9 s, total: 2min 56s
Wall time: 4min 9s


In [239]:
connects_ferc1_eia = prettyify_best_matches(
    matches_best, 
    plant_parts_true_df=inputs.plant_parts_true_df,
    steam_df=inputs.all_plants_ferc1_df,
    debug=True
)

Coverage for matches during EIA working years:
    Fuel type: 96.3%
    Tech type: 41.5%

Coverage for all steam table records during EIA working years:
    EIA matches: 56.0

Coverage for all small gen table records during EIA working years:
    EIA matches: 40.8

Coverage for all hydro table records during EIA working years:
    EIA matches: 85.6

Coverage for all pumped storage table records during EIA working years:
    EIA matches: 45.2


In [209]:
connects_ferc1_eia = prettyify_best_matches(
    matches_best, 
    plant_parts_true_df=inputs.plant_parts_true_df,
    steam_df=inputs.all_plants_ferc1_df,
)

Coverage for matches during EIA working years:
    Fuel type: 96.3%
    Tech type: 41.6%

Coverage for all steam table records during EIA working years:
    EIA matches: 56.1

Coverage for all small gen table records during EIA working years:
    EIA matches: 40.9

Coverage for all hydro table records during EIA working years:
    EIA matches: 85.6

Coverage for all pumped storage table records during EIA working years:
    EIA matches: 45.2


In [46]:
# export for connect_deprish_to_ferc1
#file_path_steam_ferc1 = pathlib.Path().cwd().parent / 'outputs' /'steam_ferc1.pkl.gz'
#inputs.steam_df.to_pickle(file_path_steam_ferc1, compression='gzip')

file_path_ferc1_eia = pathlib.Path().cwd().parent / 'outputs'/ 'ferc1_to_eia.pkl.gz'
connects_ferc1_eia.to_pickle(file_path_ferc1_eia, compression='gzip')
file_path_ferc1_eia_csv = pathlib.Path().cwd().parent / 'outputs'/ 'ferc1_to_eia.csv.gz'
matches_best.to_csv(file_path_ferc1_eia_csv)

### Investigating Comparison Feature Vectors

In [22]:
features_train.describe().T

count      mean       std           min           25%  \
plant_name           406.0  0.763008  0.115174  0.000000e+00  7.004340e-01   
net_generation_mwh   406.0  0.202667  0.376750  0.000000e+00  0.000000e+00   
capacity_mw          406.0  0.416345  0.461922  4.792842e-99  6.515007e-14   
total_fuel_cost      406.0  0.071255  0.248599  0.000000e+00  0.000000e+00   
total_mmbtu          406.0  0.125803  0.326379  0.000000e+00  0.000000e+00   
capacity_factor      406.0  0.897074  0.173541  0.000000e+00  8.667602e-01   
fuel_cost_per_mmbtu  406.0  0.426640  0.458598  0.000000e+00  0.000000e+00   
heat_rate_mmbtu_mwh  406.0  0.335532  0.436499  0.000000e+00  0.000000e+00   
fuel_type_code_pudl  406.0  0.665025  0.472564  0.000000e+00  0.000000e+00   
installation_year    406.0  0.556650  0.497393  0.000000e+00  0.000000e+00   

                               50%           75%       max  
plant_name            7.728938e-01  8.518519e-01  0.939394  
net_generation_mwh    2.415656e-47  6.574298e-02  1.000000  
capacity_mw           6.631249e-02  9.953237e-01  1.000000  
total_fuel_cost      1.053264e-220  2.143343e-37  1.000000  
total_mmbtu           0.000000e+00  2.037602e-71  1.000000  
capacity_factor       9.678165e-01  9.961894e-01  1.000000  
fuel_cost_per_mmbtu   0.000000e+00  9.381866e-01  0.999960  
heat_rate_mmbtu_mwh   0.000000e+00  8.630693e-01  1.000000  
fuel_type_code_pudl   1.000000e+00  1.000000e+00  1.000000  
installation_year     1.000000e+00  1.000000e+00  1.000000

In [23]:
features_all.describe().T

count      mean       std  min           25%  \
plant_name           105677.0  0.738466  0.112095  0.0  6.764706e-01   
net_generation_mwh   105677.0  0.128405  0.296618  0.0  0.000000e+00   
capacity_mw          105677.0  0.125476  0.289744  0.0  6.133174e-19   
total_fuel_cost      105677.0  0.109113  0.301366  0.0  0.000000e+00   
total_mmbtu          105677.0  0.102192  0.298221  0.0  0.000000e+00   
capacity_factor      105677.0  0.820448  0.275116  0.0  7.358799e-01   
fuel_cost_per_mmbtu  105677.0  0.207059  0.377680  0.0  0.000000e+00   
heat_rate_mmbtu_mwh  105677.0  0.150586  0.332506  0.0  0.000000e+00   
fuel_type_code_pudl  105677.0  0.629948  0.482821  0.0  0.000000e+00   
installation_year    105677.0  0.390586  0.487884  0.0  0.000000e+00   

                               50%           75%       max  
plant_name            7.528076e-01  8.222222e-01  1.000000  
net_generation_mwh   2.812973e-165  1.007894e-06  1.000000  
capacity_mw           1.369355e-07  2.368307e-02  1.000000  
total_fuel_cost       0.000000e+00  7.803069e-36  1.000000  
total_mmbtu           0.000000e+00  0.000000e+00  1.000000  
capacity_factor       9.555419e-01  9.944404e-01  1.000000  
fuel_cost_per_mmbtu   0.000000e+00  0.000000e+00  0.999976  
heat_rate_mmbtu_mwh   0.000000e+00  0.000000e+00  1.000000  
fuel_type_code_pudl   1.000000e+00  1.000000e+00  1.000000  
installation_year     0.000000e+00  1.000000e+00  1.000000

### Explore Classificaiton Model

In [24]:
weights = pd.DataFrame(
    data={'feature': features_all.columns,
          'weight': matcher.coefs
         })
weights

feature    weight
0           plant_name  0.000000
1   net_generation_mwh  2.511863
2          capacity_mw  1.714724
3      total_fuel_cost  0.368863
4          total_mmbtu  0.635846
5      capacity_factor -0.243250
6  fuel_cost_per_mmbtu -0.860928
7  heat_rate_mmbtu_mwh  1.278104
8  fuel_type_code_pudl  0.962714
9    installation_year  0.790152

In [25]:
def plot_all_hist(all_df,results_df,murk_df, matches_best_df, range):
    if all_df is not None:
        plt.hist(all_df['score'], 
                 range=range,
                 bins=100,
                 color="pink",
                 label='all options'
                )
    if results_df is not None:
        plt.hist(results_df['score'], 
                 range=range,
                 bins=100,
                 color="purple",
                 label='all model matches'
                )
    if matches_best_df is not None:
        plt.hist(matches_best_df['score'], 
                 range=range,
                 bins=100,
                 color="turquoise",
                 label='winning options'
                )
    if murk_df is not None:
        plt.hist(murk_df['score'], 
                 range=range,
                 bins=100,
                 color="grey",
                 label='murky wins'
                )

    plt.title(f'weighted score of comparision features')
    plt.xlabel('weighted sum')
    plt.ylabel(None)
    plt.legend()
    plt.show()

In [26]:
plot_all_hist(matcher.calc_match_stats(features_all),
              matcher.matches_model,
              matcher.murk_df,
              matches_best,
              range=None)

In [27]:
wins = (matches_best.reset_index().merge(inputs.plant_parts_df.
              reset_index()[['record_id_eia','plant_part','capacity_mw']],
              on=['record_id_eia'],
              how='left',
              suffixes=('_feature','')
             ).
        groupby(['plant_part']).
        agg({'capacity_mw':sum,
             'score': 'count'}).
        assign(count_w=lambda x: x.capacity_mw * x.score,
               percent_w=lambda x: x.count_w/x.sum()['count_w'],
               percent=lambda x: x.score/x.sum()['score'],
              )
       )

wins.round(2)

capacity_mw  score       count_w  percent_w  percent
plant_part                                                             
plant               3029528.39   5524  1.673511e+10       0.85     0.52
plant_ferc_acct         938.10      3  2.814300e+03       0.00     0.00
plant_gen            216138.90   1294  2.796837e+08       0.01     0.12
plant_prime_fuel     211279.12    334  7.056722e+07       0.00     0.03
plant_prime_mover   1042693.55   2210  2.304353e+09       0.12     0.21
plant_technology     126234.00    168  2.120731e+07       0.00     0.02
plant_unit           353356.06    991  3.501759e+08       0.02     0.09

### Improve Training Data

In [243]:
test = connects_ferc1_eia.copy()
test.loc[:,'record_id_ferc1'] = test.record_id_ferc1.astype('string')
test.loc[:,'record_id_eia'] = test.record_id_eia.astype('string')

li = [x for x in training.record_id_eia.unique() if x in test.record_id_eia.unique()]

In [244]:
for x in li:
    if len(test[test['record_id_eia']==x]) > 0:
        print(x)

1241_1_2018_plant_unit_owned_10000
1241_2_2018_plant_unit_owned_10000
1241_1_2018_plant_unit_owned_10005
1241_2_2018_plant_unit_owned_10005
1241_2018_plant_total_10000
210_2018_plant_owned_10005
210_2018_plant_owned_10000
2442_4_2018_plant_unit_owned_803
2442_5_2018_plant_unit_owned_803
2442_2018_plant_owned_24211
2451_2018_plant_owned_24211
26_1_2018_plant_unit_owned_195
2963_1_2018_plant_unit_total_15474
2964_1_2018_plant_unit_total_15474
3948_1_2018_plant_unit_owned_20521
3948_1_2018_plant_unit_owned_22053
3948_2018_plant_total_20521
3948_2018_plant_total_22053
4079_2018_plant_owned_20860
4080_2018_plant_owned_20860
4940_1_2018_plant_unit_total_15474
4941_2018_plant_owned_803
55111_2018_plant_owned_40211
55343_2018_plant_owned_24211
55380_2018_plant_owned_814
55467_2018_plant_owned_814
56456_2018_plant_owned_5860
6002_2018_plant_total_195
6008_1_2018_plant_gen_owned_803
6008_2_2018_plant_gen_owned_803
6008_3_2018_plant_gen_owned_803
6009_2018_plant_owned_814
6034_1_2018_plant_unit_o

In [118]:
test[test['record_id_ferc1']=='f1_steam_2018_12_2_1_2']
#test[test['record_id_eia']=='1005_2018_plant_total_15470']

record_id_ferc1 record_id_eia plant_name_new plant_part  \
27029  f1_steam_2018_12_2_1_2          <NA>           <NA>        NaN   

       report_year ownership plant_name_eia  plant_id_eia generator_id  \
27029         2018       NaN           <NA>           NaN          NaN   

      unit_id_pudl prime_mover_code energy_source_code_1  \
27029          NaN              NaN                  NaN   

      technology_description ferc_acct_name  utility_id_eia  \
27029                    NaN            NaN            <NA>   

       utility_id_pudl_eia true_gran appro_part_label appro_record_id_eia  \
27029                 <NA>       NaN              NaN                 NaN   

       record_count  fraction_owned ownership_dupe  plant_id_pudl_eia  \
27029           NaN             NaN            NaN               <NA>   

       total_fuel_cost_eia  net_generation_mwh_eia  capacity_mw_eia  \
27029                  NaN                     NaN              NaN   

       utility_id_ferc1  utility_id_pudl_ferc1     utility_name_ferc1  \
27029                 2                     18  ALABAMA POWER COMPANY   

       plant_id_pudl_ferc1  plant_id_ferc1  plant_name_ferc1  \
27029                11313            1236  greene county ct   

       asset_retirement_cost  avg_num_employees  capacity_factor  \
27029                    NaN                NaN         0.007139   

       capacity_mw_ferc1  capex_equipment  capex_land  capex_per_mw  \
27029             827.42      205228653.0      5838.0      259651.5   

       capex_structures  capex_total construction_type  construction_year  \
27029         9606351.0  214840842.0      conventional             1995.0   

      installation_year  net_generation_mwh_ferc1  \
27029              1996                   51747.0   

       not_water_limited_capacity_mw  opex_allowances  opex_boiler  \
27029                          740.0              NaN          NaN   

       opex_coolants  opex_electric  opex_engineering  opex_fuel  \
27029            NaN            NaN          270258.0  4633431.0   

       fuel_cost_per_mwh  opex_misc_power  opex_misc_steam  opex_nonfuel  \
27029          89.540089         171150.0          68286.0     4666252.0   

      opex_nonfuel_per_mwh  opex_operations  opex_per_mwh  opex_plant  \
27029            90.174348         453631.0         179.7   3255083.0   

       opex_production_total  opex_rents  opex_steam  opex_steam_other  \
27029              9299683.0         NaN    196778.0               NaN   

       opex_structures  opex_transfer  peak_demand_mw  plant_capability_mw  \
27029         251066.0            NaN           559.0                  NaN   

       plant_hours_connected_while_generating          plant_type  \
27029                                     NaN  combustion_turbine   

       water_limited_capacity_mw  total_fuel_cost_ferc1  total_mmbtu  \
27029                        NaN            4633599.024          0.0   

      fuel_type_code_pudl plant_name_original  ferc_license_id  \
27029                 NaN                 NaN              NaN   

       fuel_cost_per_mmbtu fuel_type  opex_maintenance  opex_total  \
27029                  inf       NaN               NaN         NaN   

       total_cost_of_plant  capex_facilities  capex_roads  \
27029                  NaN               NaN          NaN   

       net_capacity_adverse_conditions_mw  \
27029                                 NaN   

       net_capacity_favorable_conditions_mw  opex_dams  opex_generation_misc  \
27029                                   NaN        NaN                   NaN   

       opex_hydraulic  opex_misc_plant  opex_water_for_power  \
27029             NaN              NaN                   NaN   

       capex_equipment_electric  capex_equipment_misc  \
27029                       NaN                   NaN   

       capex_wheels_turbines_generators  energy_used_for_pumping_mwh  \
27029                               NaN                          NaN   

 

In [128]:
len(li)

259

In [130]:
len(li)

80

In [131]:
li

['1241_1_2018_plant_unit_owned_10000',
 '1241_1_2018_plant_unit_owned_10005',
 '1241_2018_plant_total_10000',
 '210_2018_plant_owned_10005',
 '210_2018_plant_owned_10000',
 '210_2018_plant_total_10000',
 '2442_2018_plant_owned_24211',
 '2451_2018_plant_owned_24211',
 '26_1_2018_plant_unit_owned_195',
 '2963_1_2018_plant_unit_total_15474',
 '2964_1_2018_plant_unit_total_15474',
 '3948_1_2018_plant_unit_owned_20521',
 '3948_1_2018_plant_unit_owned_22053',
 '3948_2018_plant_total_20521',
 '3948_2018_plant_total_22053',
 '4079_2018_plant_owned_20860',
 '4080_2018_plant_owned_20860',
 '4940_1_2018_plant_unit_total_15474',
 '4941_2018_plant_owned_803',
 '55111_2018_plant_owned_40211',
 '55343_2018_plant_owned_24211',
 '55380_2018_plant_owned_814',
 '55467_2018_plant_owned_814',
 '55856_2018_plant_total_40211',
 '56456_2018_plant_owned_5860',
 '6002_2018_plant_total_195',
 '6008_1_2018_plant_gen_owned_803',
 '6009_2018_plant_owned_814',
 '6034_1_2018_plant_unit_owned_5109',
 '6034_1_2018_plan

In [33]:
mul = inputs.get_plant_parts_full()

In [11]:
pudl.helpers.cleanstrings_snake(training, ['record_id_eia'])

record_id_eia           record_id_ferc1
0                   1005_2018_plant_owned_15470  f1_hydro_2018_12_144_0_1
1                  10773_2018_plant_owned_19876  f1_steam_2018_12_186_0_1
2                  10774_2018_plant_owned_19876  f1_steam_2018_12_186_3_2
3        10_gt_2018_plant_prime_mover_owned_195    f1_steam_2018_12_2_1_2
4                   1109_2018_plant_owned_19436  f1_hydro_2018_12_177_0_2
..                                          ...                       ...
469                  3456_2016_plant_total_5701   f1_steam_2016_12_49_0_3
470                 3992_2018_plant_owned_11479   f1_steam_2018_12_89_0_1
471                  470_2018_plant_total_15466  f1_steam_2018_12_145_0_4
472                 6112_2018_plant_total_15466  f1_steam_2018_12_145_2_4
473  6112_gt_2018_plant_prime_mover_total_15466  f1_steam_2018_12_145_2_5

[366 rows x 2 columns]

In [12]:
inputs.train_df

Empty DataFrame
Columns: []
Index: [(f1_hydro_2018_12_144_0_1, 1005_2018_plant_total_15470), (f1_steam_2018_12_186_0_1, 10773_2018_plant_total_19876), (f1_steam_2018_12_186_3_2, 10774_2018_plant_total_19876), (f1_steam_2018_12_2_1_2, nan), (f1_hydro_2018_12_177_0_2, 1109_2018_plant_total_19436), (f1_steam_2018_12_7_0_1, 113_1_2018_plant_unit_total_803), (f1_steam_2018_12_7_0_2, 113_1_2018_plant_unit_total_803), (f1_steam_2018_12_7_2_4, 114_2018_plant_total_803), (f1_steam_2018_12_7_0_5, 116_1_2018_plant_unit_total_803), (f1_steam_2018_12_7_1_1, 116_1_2018_plant_unit_total_803), (f1_steam_2018_12_7_3_3, 116_1_2018_plant_unit_total_803), (f1_steam_2018_12_7_3_4, 116_1_2018_plant_unit_total_803), (f1_steam_2018_12_7_3_5, nan), (f1_steam_2018_12_7_4_1, nan), (f1_steam_2018_12_7_4_3, nan), (f1_steam_2018_12_7_4_4, nan), (f1_steam_2018_12_7_4_5, nan), (f1_steam_2018_12_7_5_1, 117_1_2018_plant_unit_total_803), (f1_steam_2018_12_7_5_2, 117_2_2018_plant_unit_total_803), (f1_steam_2018_12_7_2_5, 118_1_2018_plant_gen_total_803), (f1_steam_2018_12_7_3_1, 118_1_2018_plant_gen_total_803), (f1_steam_2018_12_7_3_2, 118_1_2018_plant_gen_total_803), (f1_steam_2018_12_7_1_3, nan), (f1_steam_2018_12_7_1_4, nan), (f1_steam_2018_12_7_1_5, nan), (f1_steam_2018_12_7_2_1, nan), (f1_steam_2018_12_7_2_2, nan), (f1_steam_2018_12_7_2_3, nan), (f1_steam_2018_12_51_0_2, 1239_2018_plant_total_5860), (f1_steam_2018_12_51_0_1, nan), (f1_steam_2018_12_80_0_2, 1240_1_2018_plant_unit_total_10005), (f1_steam_2018_12_191_0_2, nan), (f1_steam_2018_12_79_2_3, 1241_1_2018_plant_unit_owned_10000), (f1_steam_2018_12_79_2_4, 1241_1_2018_plant_unit_owned_10000), (f1_steam_2018_12_80_0_3, 1241_1_2018_plant_unit_owned_10005), (f1_steam_2018_12_80_0_4, 1241_1_2018_plant_unit_owned_10005), (f1_steam_2018_12_79_2_5, 1241_2018_plant_total_10000), (f1_steam_2018_12_80_0_1, 1242_2018_plant_total_10005), (f1_steam_2018_12_191_1_1, nan), (f1_steam_2018_12_191_1_2, nan), (f1_steam_2018_12_176_1_1, 124_2018_plant_total_24211), (f1_steam_2018_12_191_1_5, 1250_2018_plant_total_22500), (f1_steam_2018_12_191_0_1, 1252_1_2018_plant_gen_total_22500), (f1_steam_2018_12_176_0_5, nan), (f1_steam_2018_12_176_0_1, 126_1_2018_plant_unit_total_24211), (f1_steam_2018_12_81_0_1, 1353_2018_plant_total_22053), (f1_steam_2018_12_8_0_1, 170_2018_plant_total_814), (f1_steam_2018_12_44_4_2, 1725_2018_plant_total_5109), (f1_steam_2018_12_44_3_4, 1728_2018_plant_total_5109), (f1_steam_2018_12_44_0_3, nan), (f1_steam_2018_12_44_3_5, nan), (f1_steam_2018_12_44_3_1, 1730_2018_plant_total_5109), (f1_steam_2018_12_44_4_4, nan), (f1_steam_2018_12_44_0_4, 1733_1_2018_plant_unit_total_5109), (f1_steam_2018_12_44_1_4, 1734_2018_plant_total_5109), (f1_steam_2018_12_44_4_5, 1735_2018_plant_total_5109), (f1_steam_2018_12_44_1_5, 1737_2018_plant_total_5109), (f1_steam_2018_12_44_2_1, 1739_2018_plant_total_5109), (f1_steam_2018_12_44_4_3, nan), (f1_steam_2018_12_44_1_1, 1740_3_2018_plant_unit_total_5109), (f1_steam_2018_12_44_5_1, 1741_2018_plant_total_5109), (f1_steam_2018_12_44_3_3, nan), (f1_steam_2018_12_44_1_2, 1743_1_2018_plant_unit_total_5109), (f1_steam_2018_12_44_2_4, 1744_2018_plant_total_5109), (f1_steam_2018_12_44_1_3, 1745_2018_plant_total_5109), (f1_steam_2018_12_44_5_2, 1746_2018_plant_total_5109), (f1_gnrt_plant_2018_12_73_0_2, nan), (f1_gnrt_plant_2018_12_73_0_3, nan), (f1_gnrt_plant_2018_12_73_0_4, nan), (f1_gnrt_plant_2018_12_73_0_6, nan), (f1_steam_2018_12_193_1_1, 1769_2018_plant_total_20847), (f1_gnrt_plant_2018_12_193_0_11, nan), (f1_gnrt_plant_2018_12_193_0_9, nan), (f1_gnrt_plant_2018_12_193_0_7, nan), (f1_gnrt_plant_2018_12_193_0_8, nan), (f1_gnrt_plant_2018_12_193_0_10, nan), (f1_gnrt_plant_2018_12_195_0_15, nan), (f1_gnrt_plant_2018_12_132_0_2, nan), (f1_steam_2018_12_100_0_2, 2050_2018_plant_total_12685), (f1_steam_2018_12_51_0_3, 2076_2018_plant_total_5860), (f1_steam_2018_12_79_0_2, nan), (f1_steam_2018_12_79_0_3, 2079_2_2018_plant_unit_total_10000), (f1_steam_2018_12_79_0_4, nan), (f1_ste

In [ ]:
inputs.prep_